## Данные

Доступны медицинские обезличенные данные (оказанные медицинские услуги) за 3 месяца :

[https://disk.yandex.ru/d/CCtVC1jKwqD1vw](https://disk.yandex.ru/d/CCtVC1jKwqD1vw)

Диагнозы брать из столбцов : МКБ10 , МКБ10_1

Описание данных : 

- Дата - дата оказания медицинской услуги
- patient_ID - уникальный ид пациента
- Дата рождения - ДР пациента (заполнено одним значением , не использовать)
- Пол - Пол пациента
- Полис - номер полиса . можно не использовать
- Код услуги - код услуги из прейскуранта
- Наименование услуги - что за услуга была оказана
- Кол-во услуг - количество оказанных услуг
- Стоимость по прайсу - стоимость услуги из прейскуранта
- Стоимость - фактическая стоимость услуги с учетом скидок (не используем)
- МКБ10 - основной код диагноза
- Зуб - номер зуба (для стоматологии)
- doc_id - ид врача (в базовой модели не используем)
- МКБ10_1 - полный список кодов диагноза
- Специальность - специальность врача (в базовой модели не используем)

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
april_1_15 = pd.read_excel('../data/april_1-15.xlsx')
april_16_30 = pd.read_excel('../data/april_16-30.xlsx')
may_1_15 = pd.read_excel('../data/may_1-15.xlsx')
may_16_31 = pd.read_excel('../data/may_16-31.xlsx')
june_1_15 = pd.read_excel('../data/june_1-15.xlsx')
june_16_30 = pd.read_excel('../data/june_16-30.xlsx')

In [ ]:
df = pd.concat([april_1_15, april_16_30, may_1_15, may_16_31, june_1_15, june_16_30], ignore_index=True)

In [4]:
df = pd.read_csv('../data/df.csv')

In [5]:
df.drop(['Unnamed: 0'], inplace=True, axis=1)

Переименую столбцы для удобства

In [6]:
df.rename(columns={'Дата':'date_visit', 'Дата рождения':'birth', 'Пол':'gender', 'Полис':'insurance', 'Код услуги':'service_code', 
       'Наименование услуги':'service_name', 'Кол-во услуг':'service_count', 'Стоимость по прайсу':'price', 
       'Стоимость':'price_sale', 'МКБ10':'main_diagnosis', 'Зуб':'tooth', 'МКБ10_1':'all_diagnosis_codes', 'Специальность':'specialization'}, 
         inplace=True)

In [7]:
df.gender = df.gender.astype('category')
df.main_diagnosis = df.main_diagnosis.astype('category')
df.date_visit = pd.to_datetime(df.date_visit)

In [11]:
df = df.drop(['birth', 'insurance', 'tooth', 'doc_id'], axis=1)

Создал новый датафрейм (отсортированный по дате) для манипуляции над ним, в будущем не пригодилось создавать именно новый обьект, можно было и с одним и тем же работать

In [12]:
df_sorted = df.copy()

In [13]:
df_sorted.date_visit = pd.to_datetime(df_sorted.date_visit, format='%Y-%m-%d')

In [14]:
df_sorted = df_sorted.sort_values(['date_visit'])

In [16]:
df_sorted.patient_ID.dropna(inplace=True)

In [17]:
df_sorted = df_sorted[(df_sorted.patient_ID != 0) | (df_sorted.service_name != 'Внесение аванса')]

In [18]:
df_sorted.index = range(df_sorted.shape[0])

Сгруппировал по пациентам

In [19]:
patient_groups = df_sorted.groupby('patient_ID')

In [22]:
df_sorted.drop(['service_name','all_diagnosis_codes',  'specialization'], axis='columns', inplace=True)

Вот именно над следующей ячейкой я работал очень долго

Моя идея заключалась в том, чтобы сначала разбить выборку на кейсы а потом представить каждый кейс единой записью в обучающей выборке 

In [23]:
cases = []
counter = 0
for name, group in patient_groups:
    group.date_visit = pd.to_datetime(group.date_visit, format='%Y-%m-%d')
    group = group.sort_values(['date_visit'])
    group['next_visit'] = group['date_visit'].shift(-1)
    group['date_diff'] = group['next_visit'] - group['date_visit']
    group['date_diff'] = group['date_diff'].fillna(timedelta(days=0))
    group = group.reset_index(drop=True)
    group['condition'] = abs((group['date_diff']).dt.days) > 14
    indexes = []
            
    
    if True in group.condition:
        indexes = indexes + group[group.condition == True].index.values.tolist()
        if list(group['condition'])[-1] == False:
            indexes.append(group.index[-1])
        
        cases.append(group[:indexes[0]+1])
        # print(cases[-1])
        # print("***************")
        # print(indexes)
        for i in range(0, len(indexes)-1):
            cases.append(group[indexes[i]+1:indexes[i+1] + 1])
            # print(cases[-1])
            # print("----------------")
    else:
        cases.append(group)
        # print(cases[-1])
    counter = counter + 1
    if counter % 25000 == 0:
        print(counter)
        
        

25000
50000
75000
100000
125000
150000


In [129]:
for case in cases:
    case.drop(['next_visit', 'date_diff', 'condition', 'date_visit', 'specialization'], axis='columns', inplace=True)

Здесь я делаю разбивку кода диагноза не несколько частей, думал что буду этим пользоваться, так как рассчитывал что это мне поможет упростить работу с категорией "диагноз"

In [24]:
import re

In [25]:
# prog = re.compile(pattern)
# result = prog.match(string)
def get_letter(string):
    if re.match( r'[A-Z]\d\d[.]\d', string=string):
        return string[0]
    else:
        return '0'
def get_digit_1(string):
    if re.match( r'[A-Z]\d\d[.]\d', string=string):
        return string[1:3]
    else:
        return '0'
def get_digit_2(string):
    if re.match( r'[A-Z]\d\d[.]\d', string=string):
        return string[4]
    else:
        return '0'

In [26]:
diagnoses = df_sorted.main_diagnosis.unique()

Здесь вытаскиваю валидные коды диагнозов, потому что в данных часто встречаются ошибки: записи о взятии анализов зачастую заполнены неправильно и я решил их в начале вообще убрать

In [27]:
right_diagnoses = []
for diag in diagnoses:
    if re.match(r'[A-Z]\d\d[.]\d', string=str(diag)):
        right_diagnoses.append(diag)

In [28]:
for i in range(len(right_diagnoses)):
    right_diagnoses[i] = str(right_diagnoses[i])[:3]

In [29]:
cat_feat = pd.Series(right_diagnoses).unique().tolist()

In [178]:
df_sorted['shorten_diag'] = df_sorted['main_diagnosis'].map(shorten)

Следующая функция сокращает код диагноза так как я решил в бэйзлайне обойтись one-hot кодированием, а для этого нужно было сократить количество переменных

Плюс к этому последняя цифра в коде не несет очень много информации в себе

In [30]:
def shorten(value):
    if re.match(r'[A-Z]\d\d[.]\d', string=str(value)):
        return str(value)[:3]
    elif re.match(r'[A-Z]\d\d', string=str(value)):
        return value

In [30]:
tmp['shorten_diag'] = tmp['main_diagnosis'].map(shorten)

In [58]:
tmp

,date_visit,patient_ID,gender,service_code,service_count,price,price_sale,main_diagnosis,next_visit,date_diff,condition,shorten_diag,0,match
0,2019-05-23,1150188.0,1.0,2030001,1.000000,423,423,NaN,2019-05-28,5 days,False,0,1,NaN
1,2019-05-28,1150188.0,1.0,4102022,1.000000,1759,1759,NaN,2019-05-28,0 days,False,0,1,NaN
2,2019-05-28,1150188.0,1.0,1060001,1.000000,1575,1575,M54.5,2019-05-28,0 days,False,M54,0,True
3,2019-05-28,1150188.0,1.0,4090070,1.000000,1453,1453,NaN,2019-05-28,0 days,False,0,1,NaN
4,2019-05-28,1150188.0,1.0,4106007,1.000000,720,720,NaN,2019-05-29,1 days,False,0,1,NaN
5,2019-05-29,1150188.0,1.0,4020078,1.000000,2150,2150,N84.0,2019-06-19,21 days,True,N84,0,True


In [56]:
tmp['match'] = tmp.main_diagnosis.str.match('[A-Z]\d\d([.]\d)?')

In [61]:
tmp_cols = tmp.pivot(columns=['shorten_diag'], values='match').fillna('0')

In [68]:
tmp_cols

shorten_diag,0,M54,N84
0,0,0,0
1,0,0,0
2,0,True,0
3,0,0,0
4,0,0,0
5,0,0,True


In [31]:
cases_tmp = cases

In [40]:
print(re.match(r'(?:\d*\.\d+|\d+)', '222222,311111'))

<re.Match object; span=(0, 6), match='222222'>


In [41]:
len(cases_tmp)

242120

In [51]:
tmp = cases[1]

In [55]:
tmp.pivot(columns=['shorten_diag'], values='match')

shorten_diag,NaN,H52,I11,M42,Z10
6,NaN,NaN,NaN,True,NaN
7,NaN,NaN,NaN,NaN,True
8,NaN,NaN,NaN,NaN,True
9,NaN,NaN,NaN,True,NaN
10,NaN,NaN,NaN,NaN,True
11,NaN,NaN,NaN,NaN,True
12,NaN,NaN,NaN,NaN,True
13,NaN,NaN,NaN,NaN,True
14,NaN,NaN,NaN,NaN,True
15,NaN,NaN,NaN,NaN,NaN


In [61]:
cat_feat_df = pd.Series(cat_feat, name='diagnose')

В следующей ячейке происходит то самое разделение каждого кейса на отдельные записи для обучающей выборки

Над этим кодом по факту тоже очень много времени работал, и сейчас не знаю, кажется такое представление данных для корректного решения задачи совсем не подходит

In [79]:
dicts = []
counter = 0
checks = []
for case in cases:
    case['shorten_diag'] = case['main_diagnosis'].map(shorten)
    case['match'] = case.main_diagnosis.str.match('[A-Z]\d\d([.]\d)?')
    tmp_case = case.pivot(columns=['shorten_diag'], values='match')
    
    if np.nan in tmp_case.columns:
        tmp_case.drop([np.nan], axis=1, inplace=True)
    
    cols = tmp_case.columns
    dict_ = {x:[] for x in cols}
    dict_['money'] = []
    dict_['patient_ID'] = []
    dict_['gender'] = []
    
    for col in cols:
        dict_[col].append(1)
        
    # for feat in cat_feat:
    #     if feat not in dict_.keys():
    #         dict_[feat] = []
    #         dict_[feat].append(0)
            
    case['check_price'] = case.price.str.match(r'(?:\d*\.\d+|\d+)')
    case = case[case['check_price']==True]
    case.index = range(case.shape[0])
    case.price = case.price.astype('float')
    # print(case)
    try:
        dict_['patient_ID'].append(case.patient_ID[0])
        dict_['money'].append(case.price.sum())
        dict_['gender'].append(case.gender[0])
        dicts.append(dict_)
    except:
        print(counter)
    checks.append(pd.DataFrame(dict_))
    counter = counter + 1
    if counter == 1:
        print(datetime.now())
    if counter % 1000 == 0:
        print(datetime.now())
        

2022-01-30 23:49:54.689689
986
2022-01-30 23:50:05.524206
2022-01-30 23:50:15.958257
2022-01-30 23:50:26.418832
2022-01-30 23:50:36.804799
2022-01-30 23:50:47.209751
2022-01-30 23:50:57.594203
2022-01-30 23:51:07.970220
7605
7656
2022-01-30 23:51:18.407878
2022-01-30 23:51:28.793622
2022-01-30 23:51:39.184621
2022-01-30 23:51:50.094004
2022-01-30 23:52:00.976626
2022-01-30 23:52:11.799704
2022-01-30 23:52:22.672909
14976
2022-01-30 23:52:33.530857
15561
2022-01-30 23:52:44.414076
2022-01-30 23:52:55.313644
2022-01-30 23:53:06.166716
2022-01-30 23:53:16.992203
2022-01-30 23:53:27.864578
2022-01-30 23:53:38.729061
21022
2022-01-30 23:53:49.558627
2022-01-30 23:54:00.462657
2022-01-30 23:54:11.353555
2022-01-30 23:54:22.665672
25119
2022-01-30 23:54:33.613018
2022-01-30 23:54:44.452842
2022-01-30 23:54:55.293854
2022-01-30 23:55:06.165930
2022-01-30 23:55:17.005618
30941
2022-01-30 23:55:27.872806
31935
2022-01-30 23:55:38.735086
2022-01-30 23:55:49.566620
33334
2022-01-30 23:56:00.549050

In [80]:
concated = pd.concat(checks)
concated.to_csv('../data/onehote.csv')

In [90]:
concated.fillna(0, inplace=True)

In [95]:
set(concated.columns) -  set(cat_feat) 

{'A38',
 'A46',
 'A55',
 'A64',
 'A90',
 'B07',
 'B09',
 'B49',
 'B54',
 'B64',
 'B67',
 'B75',
 'B80',
 'B86',
 'B89',
 'B99',
 'C01',
 'C07',
 'C12',
 'C19',
 'C20',
 'C23',
 'C37',
 'C56',
 'C61',
 'C64',
 'C65',
 'C66',
 'C73',
 'C97',
 'D24',
 'D27',
 'D34',
 'D45',
 'D65',
 'D70',
 'D71',
 'D77',
 'E02',
 'E43',
 'E45',
 'E46',
 'E52',
 'E58',
 'E65',
 'E68',
 'E90',
 'F03',
 'F09',
 'F24',
 'F39',
 'F83',
 'F88',
 'F99',
 'G01',
 'G09',
 'G20',
 'G26',
 'G35',
 'G64',
 'G92',
 'G98',
 'H46',
 'H55',
 'H71',
 'H82',
 'I10',
 'I38',
 'I64',
 'I66',
 'I99',
 'J00',
 'J13',
 'J22',
 'J36',
 'J40',
 'J42',
 'J46',
 'J47',
 'J90',
 'J91',
 'K20',
 'K30',
 'K36',
 'K37',
 'L00',
 'L14',
 'L22',
 'L26',
 'L42',
 'L52',
 'L80',
 'L82',
 'L84',
 'L86',
 'L97',
 'N10',
 'N12',
 'N19',
 'N23',
 'N26',
 'N40',
 'N44',
 'N46',
 'N47',
 'N61',
 'N62',
 'N63',
 'N72',
 'N86',
 'N96',
 'P38',
 'R05',
 'R11',
 'R13',
 'R14',
 'R15',
 'R17',
 'R18',
 'R21',
 'R31',
 'R32',
 'R33',
 'R35',
 'R42',


In [104]:
concated.money.min()

0.0

In [133]:
data_frames = [pd.DataFrame(x) for x in dicts]

In [142]:
data_frames[7]

,E22,E34,money,patient_ID,gender,J01,S03,E07,S30,J30,...,D04,Z72,B43,C76,I01,W64,X32,C95,L67,E84
0,1,1,5578.0,1150218.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [137]:
df_wide = pd.concat(data_frames)

In [140]:
df_wide.describe()

,M54,N84,money,patient_ID,gender,J01,S03,E07,S30,J30,...,D04,Z72,B43,C76,I01,W64,X32,C95,L67,E84
count,10.000000,10.000000,10.000000,1.000000e+01,10.000000,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
mean,0.100000,0.100000,8344.400000,1.150204e+06,0.900000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.316228,0.316228,12796.806869,1.202035e+01,0.316228,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,1687.000000,1.150188e+06,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,2774.750000,1.150198e+06,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,4286.500000,1.150204e+06,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.000000,0.000000,7243.750000,1.150215e+06,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.000000,1.000000,44164.000000,1.150222e+06,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Код ниже - попытки заменить ошибочные данные в полях цена и диагноз

В экселе я заметил, что некоторые значения все таки не пропущены, они только лежат совсем в другом месте и попытался несколькими способами их поставить, но
позже понял, что это снова слишком долго и может вообще не принести никакой пользы

In [185]:
price_check = df_sorted.price.astype('float32')

ValueError: could not convert string to float: ' Кратинин'

In [186]:
df_sorted[df_sorted.price == ' Кратинин']

,date_visit,patient_ID,gender,service_code,service_name,service_count,price,price_sale,main_diagnosis,all_diagnosis_codes,specialization
2605377,2019-06-25,1153795.0,0.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
121566,2019-04-04,1318754.0,0.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
1324987,2019-05-14,1381349.0,1.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
778296,2019-04-23,1551863.0,0.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
1636417,2019-05-23,1784162.0,0.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
...,...,...,...,...,...,...,...,...,...,...,...
2507527,2019-06-21,5495234.0,1.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
2302494,2019-06-14,5495271.0,1.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
2634530,2019-06-26,5500009.0,1.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий
2287236,2019-06-13,5501625.0,1.0,3200036,Биохимический анализ крови (Общий белок,Мочевина,Кратинин,Билирубин общий,Аланинаминотрансфераза (АЛТ),Железо,Холестерин общий


In [188]:
df_sc = df_sorted.copy()

In [205]:
df_sc = df_sc[df_sc.price != ' Кратинин']
df_sc = df_sc[df_sc.price != ' Тромбиновое время']
df_sc = df_sc[df_sc.price != ' Группа крови + Резус-фактор']
df_sc = df_sc[df_sc.price != ' Поверхностный антиген вируса гепатита В (австралийский антиген, HbsAg))']
df_sc = df_sc[df_sc.price != ' t4 Лещина обыкновенная (орешник)']
df_sc = df_sc[df_sc.price != 'направлений (с учетом НДС)']
df_sc = df_sc[df_sc.price != ' ДНК микоплазмы (Mycoplasma genitalium)']
df_sc = df_sc[df_sc.price != ' С-реактивный белок']
df_sc = df_sc[df_sc.price != ' Холестерин низкой плотности (ЛПНП, HDL)']


In [206]:
price_check = df_sc.price.astype('float32')

ValueError: could not convert string to float: ' m3 Aspergillus fumigatus, m5 Candida albicans'

In [209]:
errors = []
for idx, row in df_sc.iterrows():
    try:
        value = float(row.price)
    except:
        errors.append(row.price)        

In [210]:
len(errors)

377

In [213]:
len(pd.Series(errors).unique())

39

Потеряна некоторая часть данных, точнее они указаны неправильно
Эти потери - неправильное расположение данных, связаны они с анализами, они указаны в таблицах не в тех ячейках (где-то даже не указаны вообще)

Идем их искать!

In [214]:
april_1_15 = pd.read_excel('../data/april_1-15.xlsx')
april_16_30 = pd.read_excel('../data/april_16-30.xlsx')
may_1_15 = pd.read_excel('../data/may_1-15.xlsx')
may_16_31 = pd.read_excel('../data/may_16-31.xlsx')
june_1_15 = pd.read_excel('../data/june_1-15.xlsx')
june_16_30 = pd.read_excel('../data/june_16-30.xlsx')

KeyboardInterrupt: 

In [215]:
dasha = pd.concat([april_1_15, april_16_30, may_1_15, may_16_31, june_1_15, june_16_30], ignore_index=True)

In [216]:
dasha.rename(columns={'Дата':'date', 'Дата рождения':'birth', 'Пол':'gender', 'Полис':'insurance', 'Код услуги':'service_code', 
       'Наименование услуги':'service_name', 'Кол-во услуг':'service_count', 'Стоимость по прайсу':'price', 
       'Стоимость':'price_sale', 'МКБ10':'main_diagnosis', 'Зуб':'tooth', 'МКБ10_1':'all_diagnosis_codes', 'Специальность':'specialization'}, 
         inplace=True)

In [219]:
errors_codes = {'errors': [], 'codes':[]}
for idx, row in df_sc.iterrows():
    try:
        value = float(row.price)
    except:
        errors_codes['errors'].append(row.price)
        errors_codes['codes'].append(row.service_code)

In [235]:
comparer = pd.DataFrame(errors_codes).drop_duplicates()

In [238]:
comparer.rename(columns={'codes': 'service_code'}, inplace=True)

In [223]:
df_sorted[df_sorted.service_code == '3210046']

,date_visit,patient_ID,gender,service_code,service_name,service_count,price,price_sale,main_diagnosis,all_diagnosis_codes,specialization
1419916,2019-05-16,3431610.0,1.0,3210046,"ImmunoCAP. Пыльца сорных трав, микст wx2 (w2 А...",w6 Полынь обыкновенная (чернобыльник),w9 Подорожник ланцетолистный,w10 Марь белая,w15 Лебеда),КДЦП,212597
1199441,2019-05-08,4941699.0,1.0,3210046,"ImmunoCAP. Пыльца сорных трав, микст wx2 (w2 А...",w6 Полынь обыкновенная (чернобыльник),w9 Подорожник ланцетолистный,w10 Марь белая,w15 Лебеда),КДЦП,212597
1199442,2019-05-08,4941699.0,1.0,3210046,"ImmunoCAP. Пыльца сорных трав, микст wx2 (w2 А...",w6 Полынь обыкновенная (чернобыльник),w9 Подорожник ланцетолистный,w10 Марь белая,w15 Лебеда),КДЦП,212597


In [269]:
dasha[dasha.service_code == 3200048].iloc[:,4:15].doc_id.mean()

1819.4

In [239]:
merge_res = dasha.merge(right=comparer, how='right', on='service_code')

In [243]:
dasha[dasha.service_code.isin(list(comparer.service_code))]

,date,patient_ID,birth,gender,insurance,service_code,service_name,service_count,price,price_sale,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26


In [244]:
comparer

,errors,service_code
0,"m3 Aspergillus fumigatus, m5 Candida albicans",3210036
1,g5 Райграс пастбищный/Плевел,3210044
2,Антиген CA 19-9,3200052
3,уточнающий диагноз и оценка эффективности леч...,3200007
4,d2 Клещ домашней пыли/Dermatophagoides farinae,3210035
6,Мочевина,3200044
7,Пепсиноген I/Пепсиноген II (соотношение),3200048
9,d1 Клещ домашней пыли/Dermatophagoides pterоn...,3210061
12,Креатинин,3200035
13,Инсулин,3200047


In [272]:
'''
3210036 : 1340
3200052 : 2105
3210044 : 1327
3200007 : 2300
3210035 : 1255
3200044 : 316
3200048 : 1819 
'''

'\n3210036 : 1340\n3200052 : 2105\n3210044 : 1327\n3200007 : 2300\n3210035 : 1255\n3200044 : 316\n3200048 : 1819 \n'